In [4]:
import re
import swifter
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv("data/sample_submission.csv")

In [ ]:
# 결측치 확인
def check_missing_col(df):
    missing_col = []
    for col in df.columns:
        missing_values = sum(df[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다')
            missing_col.append([col, df[col].dtype])
        if missing_col == []:
            print('결측치가 존재하지 않습니다')
        return missing_col

In [ ]:
def clean_text(texts):
    corpus = []
    for i in range(len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
        review = re.sub(r'\d+','', review)#숫자 제거
        review = review.lower() #소문자 변환
        review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'\s+', ' ', review) #spaces 제거
        review = re.sub(r"^\s+", '', review) #space from start 제거
        review = re.sub(r'\s+$', '', review) #space from the end 제거
        review = re.sub(r'_', ' ', review) #space from the end 제거
        corpus.append(review) 
    return corpus

In [ ]:
missing_col = check_missing_col(train)

In [ ]:
missing_col = check_missing_col(test)

In [ ]:
wlem = nltk.WordNetLemmatizer()
stopwords_list = stopwords.words('english')

In [ ]:
train['text'] = clean_text(train['text'])
test['text'] = clean_text(test['text'])

In [ ]:
train['tokenized'] = train['text'].apply(nltk.word_tokenize)
train['tokenized'] = train['tokenized'].apply(lambda x: [item for item in x if item not in stopwords_list])
train['text_re'] = train['tokenized'].apply(lambda x:' '.join(x))

In [ ]:
test['tokenized'] = test['text'].apply(nltk.word_tokenize)
test['tokenized'] = test['tokenized'].apply(lambda x: [item for item in x if item not in stopwords_list])
test['text_re'] = test['tokenized'].apply(lambda x:' '.join(x))

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(np.array(train["text"]))
X_test = vectorizer.transform(test["text"])

In [ ]:
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(train['text_re'])
# X_test = vectorizer.transform(test['text_re'])

In [ ]:
X_test = vectorizer.transform(test['text_re'])

In [6]:
y = train['target']

In [8]:
# logistic regression
model = LogisticRegression(max_iter=500)
model.fit(X, y)

LogisticRegression(max_iter=500)

In [9]:
pred = model.predict(X_test)
submission['target'] = pred
submission.to_csv("data/submission0408(2).csv", index = False)

In [ ]:
# 앙상블 모델
# 파라미터 튜닝

# https://dacon.io/competitions/official/235670/codeshare/1893?page=1&dtype=recent